In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [15]:
from static_variables.dataset_variables import CholecInstanceSegVariables 
seq_to_split_dict = CholecInstanceSegVariables.seq_to_split_dict  
from utils.save_files import save_to_json
from os.path import join
import os

In [16]:
instance_full_seqs = [seq for seq in seq_to_split_dict.keys() if seq.split('_')[-1] =='full']

In [17]:
tool_annotations_folder_path = 'C:/Users/tal22/Downloads/datasets/Cholec80/tool_annotations/'
save_tool_annotations_json_folder_path =  'C:/Users/tal22/Documents/repositories/generate_binary_and_instance_masks_for_cholecseg8k/datasets/quality_control_image_labels/cholec_80_classification_labels'


In [18]:
def convert_tool_txt_to_tool_json(txt_path, json_save_path):
    
    seq_name = os.path.basename(json_save_path) 
    seq_name = seq_name.split('.')[0]
    
    with open(txt_path, 'r') as file:
        data = file.read()
    
    # Split the data into lines
    lines = data.split('\n')

    # Extract the first line as headers (tool names)
    headers = lines[0].split('\t')[1:-1]  # Ignore 'Frame' and 'SpecimenBag'
    headers = [class_name.lower() for class_name in headers]

    # Initialize the dictionary
    tool_dict = {}

    # Iterate over the remaining lines
    for line in lines[1:]:
        # Split the line into components
        parts = line.split('\t')
        
        if parts == ['']:
            continue
        
        # Get the frame number (as integer divided by 25)
        frame_index = str(int(parts[0]) // 25).zfill(6)
        
        # Initialize the list of tools for this frame
        tools_present = []
        
        # Check each tool's value; if it's '1', add the tool's name to the list
        for tool_index, value in enumerate(parts[1:-1]):  # Ignore 'SpecimenBag' value
            if int(value) == 1:
                tools_present.append(headers[tool_index])
        
        # Assign the list of tools to the frame index in the dictionary
        dict_key = f't50_{seq_name}_{frame_index}' 
        tool_dict[dict_key] = tools_present
    
    save_to_json(tool_dict, json_save_path)    

In [24]:
all_cholec_80_seqs = [str(seq_num).zfill(2) for seq_num in range(1,81) ]

In [25]:
all_cholec_80_seqs

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80']

In [26]:
for seq_id in all_cholec_80_seqs:
    txt_name = f'video{seq_id}-tool.txt'
    json_name = f'VID{seq_id}.json'
    txt_path = join(tool_annotations_folder_path, txt_name)
    json_save_path = join(save_tool_annotations_json_folder_path, json_name)
    convert_tool_txt_to_tool_json(txt_path, json_save_path)


In [7]:
# for seq in instance_full_seqs:
#     seq_id = seq.split('_')[0][3:]
#     txt_name = f'video{seq_id}-tool.txt'
#     json_name = f'VID{seq_id}.json'
#     txt_path = join(tool_annotations_folder_path, txt_name)
#     json_save_path = join(save_tool_annotations_json_folder_path, json_name)
#     convert_tool_txt_to_tool_json(txt_path, json_save_path)
    
    
   

### Convert my labels to just classification labels. 

We need to get all the class names present then turn that to a tuple store that as a json with the file names. Then compare jsons

In [28]:
import sys
sys.path.append('../..')

In [59]:
from static_variables.dataset_variables import CholecInstanceSegVariables 
seq_to_split_dict = CholecInstanceSegVariables.seq_to_split_dict  
from utils.save_files import save_to_json
from utils.read_files import read_from_json
from os.path import join
import os

In [60]:
dataset_partition_path =  'C:/Users/tal22/Documents/repositories/generate_binary_and_instance_masks_for_cholecseg8k/datasets/instance_cholec_v6_during_quality_control/Instance_CholecT50_sparse'
save_dataset_partition_classification_labels_path =  'C:/Users/tal22/Documents/repositories/generate_binary_and_instance_masks_for_cholecseg8k/datasets/quality_control_image_labels/classification_instance_cholec_T50_sparse'

In [61]:
def generate_classification_labels_for_seq(seq_path):
    ann_dir_path = join(seq_path, 'ann_dir')
    
    classification_labels_for_seq = {}
    
    for ann in os.listdir(ann_dir_path):
        ann_name_no_ext = ann.split('.')[0]
        ann_path = join(ann_dir_path, ann)
        
        data = read_from_json(ann_path) 
        classes_in_img = []
        for instance_info in data['shapes']:
            class_name = instance_info['label']
            if class_name not in classes_in_img:
                classes_in_img.append(class_name)
                

        classification_labels_for_seq[ann_name_no_ext] = classes_in_img 
    
    return  classification_labels_for_seq      
        
          
    

In [62]:
def generate_and_save_classification_labels_for_dataset_partition(dataset_path, save_classification_path):
    for seq_name in os.listdir(dataset_path):
        seq_path = join(dataset_path, seq_name)
        classification_labels_for_seq = generate_classification_labels_for_seq(seq_path)
        save_to_json(classification_labels_for_seq, f'{join(save_classification_path, seq_name)}.json' )
    

In [63]:
generate_and_save_classification_labels_for_dataset_partition(dataset_partition_path, 
                                                                    save_dataset_partition_classification_labels_path)

## Make a list of if there are complete subsets.  

In [64]:
def get_the_labels_with_differences(cholec_80_classification_dir, dataset_patition_classification_dir,  log_save_dir):
    seq_ids = [seq.split('_')[0][3:] for seq in  os.listdir(dataset_patition_classification_dir) ]
    for classification_ann_name in os.listdir(dataset_patition_classification_dir):
        seq_id = classification_ann_name.split('_')[0][3:]
        if int(seq_id) <= 80:  #we only have 80 for quality control
            cholec_80_classification_path = join(cholec_80_classification_dir, f'VID{seq_id}.json')
            dataset_partition_classification_path = join(dataset_patition_classification_dir, classification_ann_name)
            

            cholec_80_seq_classification_labels = read_from_json(cholec_80_classification_path)
            dataset_partition_seq_classification_labels = read_from_json(dataset_partition_classification_path)

            diff_for_seq = {}
            for label_name, cholec80_classification_labels_for_a_label in cholec_80_seq_classification_labels.items():
                if label_name in dataset_partition_seq_classification_labels:
                    dataset_partition_classification_labels_for_a_label =  dataset_partition_seq_classification_labels[label_name]
                    set_cholec80 = set(cholec80_classification_labels_for_a_label)
                    set_cholect50_full = set(dataset_partition_classification_labels_for_a_label)

                    if not set_cholec80.issubset(set_cholect50_full):
                        diff_for_seq[label_name] = list(set_cholec80 - set_cholect50_full)
            
            save_to_json(diff_for_seq, join(log_save_dir, classification_ann_name) )
            
                # print(f'{label_name} - missing labels - {set_cholec80 - set_cholect50_full}')

In [65]:
cholec_80_classification_dir = 'C:/Users/tal22/Documents/repositories/generate_binary_and_instance_masks_for_cholecseg8k/datasets/quality_control_image_labels/cholec_80_classification_labels'
dataset_patition_classification_dir = 'C:/Users/tal22/Documents/repositories/generate_binary_and_instance_masks_for_cholecseg8k/datasets/quality_control_image_labels/classification_instance_cholec_T50_sparse'

log_save_dir = '../../logs/cholec80_quality_control_cholecT50_sparse_second'

seq_names = os.listdir(dataset_patition_classification_dir) 

In [66]:
get_the_labels_with_differences(cholec_80_classification_dir, dataset_patition_classification_dir,  log_save_dir)